In [ ]:
from ipynb.fs.full.utils import *
from ipynb.fs.full.face_collections import *

In [ ]:
def detect_faces_api(photo_bytes):
    client=boto3.client('rekognition')
    try:
        response = client.detect_faces(
            Image={'Bytes': photo_bytes},                                                                       
            Attributes = ["ALL"]
        )
    except Exception as e:
        print("Couldn't detect faces in %s.", repr(e))
        raise
    else:
        return {'face_details': response['FaceDetails'], 'face_count': len(response['FaceDetails'])}

In [ ]:
def detect_faces_crop(photo_name, photo_bytes):
    try:
        faces = detect_faces_api(photo_bytes)

        bounding_boxes = []
        for face in faces['face_details']:
            bounding_boxes.append(face['BoundingBox'])
        cropped_photos = crop_bounding_boxes(photo_name, photo_bytes, bounding_boxes)
        
        return cropped_photos, bounding_boxes
    except Exception as e:
        print(repr(e))

In [ ]:
# This class was taken from the AWS demos found in:
# - https://docs.aws.amazon.com/rekognition/latest/dg/api-video-roles.html#api-video-roles-all-topics
# - https://docs.aws.amazon.com/rekognition/latest/dg/video-analyzing-with-sqs.html
# - https://docs.aws.amazon.com/rekognition/latest/dg/faces-sqs-video.html
# - https://docs.aws.amazon.com/rekognition/latest/dg/procedure-person-search-videos.html

class VideoDetect:
    
    # Initialize class variables
    rek = boto3.client('rekognition')
    sqs = boto3.client('sqs')
    sns = boto3.client('sns')
    roleArn = ''
    bucket = ''
    video = ''
    jobId = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video):    
        self.roleArn = role
        self.bucket = bucket
        self.video = video

    def GetSQSMessageSuccess(self):
        """
        Function for getting SQS responses.
        Constantly monitors the Rekognition SQS queue for a "Job Success" message.
        jobFound is set to true when the started Rekognition job id is found in the response queue
        """
        jobFound = False
        succeeded = False
    
        dotLine=0
        while jobFound == False:
            # check sqs responses
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                          MaxNumberOfMessages=10)

            if sqsResponse:
                if 'Messages' not in sqsResponse:
                    if dotLine<20:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0    
                    sys.stdout.flush()
                    # time.sleep(1)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    
                    
                    if rekMessage['JobId'] == self.startJobId:
                        print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                       ReceiptHandle=message['ReceiptHandle'])
                    else:
                        print("Job didn't match:" +
                              str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                   ReceiptHandle=message['ReceiptHandle'])


        return succeeded
    
    def StartFaceDetection(self):
        response = self.rek.start_face_detection(
            Video = {
                'S3Object': {
                    'Bucket': self.bucket, 
                    'Name': self.video
                }
            },                                                                                                 
            FaceAttributes = "ALL",  
            NotificationChannel = {
                'RoleArn': self.roleArn, 
                'SNSTopicArn': self.snsTopicArn
            }
        )

        self.startJobId=response['JobId']
        print('Start Job Id: ' + self.startJobId)
        
    def StartFaceSearchCollection(self, collection_id):
        response = self.rek.start_face_search(Video={'S3Object':{'Bucket':self.bucket,'Name':self.video}},
            CollectionId = collection_id,
            NotificationChannel = {'RoleArn':self.roleArn, 'SNSTopicArn':self.snsTopicArn})
        
        self.startJobId=response['JobId']
        
        print('Start Job Id: ' + self.startJobId)

    def GetFaceDetectionResults(self):
        maxResults = 1000
        paginationToken = ''
        finished = False
        
        faces = {}
        while finished == False:
            response = self.rek.get_face_detection(JobId=self.startJobId,
                                            MaxResults=maxResults,
                                            NextToken=paginationToken)
            print('-------------------------------------------------------------')
            print('Codec: ' + response['VideoMetadata']['Codec'])
            print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
            print('Format: ' + response['VideoMetadata']['Format'])
            print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
            print()
            
            print(response['Faces'])
            print('-------------------------------------------------------------')

            for faceDetection in response['Faces']:
                timestamp = str(faceDetection['Timestamp'])
                face = faceDetection['Face']
                # face = {
                #     'BoundingBox': faceDetection['Face']['BoundingBox'],
                #     'Landmarks': faceDetection['Face']['BoundingBox'],
                #     'Pose': faceDetection['Face']['Pose'],
                #     'Quality': faceDetection['Face']['Quality'],
                #     'Confidence': faceDetection['Face']['Confidence']    
                # }
                if faces.get(timestamp, None) is None:
                    faces[timestamp] = [face]
                else:
                    faces[timestamp].append(face)

            if 'NextToken' in response:
                paginationToken = response['NextToken']
            else:
                finished = True
                
        return faces, response['VideoMetadata']['FrameRate']
    
    def GetFaceSearchCollectionResults(self):
        maxResults = 10
        paginationToken = ''

        finished = False

        faces = {}
        while finished == False:
            response = self.rek.get_face_search(JobId=self.startJobId,
                                        MaxResults=maxResults,
                                        NextToken=paginationToken)

            print(response['VideoMetadata']['Codec'])
            print(str(response['VideoMetadata']['DurationMillis']))
            print(response['VideoMetadata']['Format'])
            print(response['VideoMetadata']['FrameRate'])
            
            persons = {}
            for personMatch in response['Persons']:
                timestamp = str(personMatch['Timestamp'])
                p_index = str(personMatch['Person']['Index'])
                print('Person Index: ' + p_index)
                print('Timestamp: ' + timestamp)
                
                max_similarity = 0.0
                max_face_id = None
                if ('FaceMatches' in personMatch):
                    for faceMatch in personMatch['FaceMatches']:
                        
                        if faceMatch['Similarity'] > max_similarity:
                            max_similarity = faceMatch['Similarity']
                            max_face_id = faceMatch['Face']['FaceId']
                    
                if max_face_id is not None:
                    print('Face ID: ' + max_face_id)
                    print('Similarity: ' + str(max_similarity))
                    
                persons[p_index] = {
                    'Details': personMatch['Person']['Face'],
                    'FaceId': max_face_id,
                    'Similarity': max_similarity
                }
                
                
                if faces.get(timestamp, None) is None:
                    faces[timestamp] = [persons.copy()]
                else:
                    faces[timestamp].append(persons)
                    
            if 'NextToken' in response:
                paginationToken = response['NextToken']
            else:
                finished = True
                
        return faces, response['VideoMetadata']['FrameRate']
        
    def CreateTopicandQueue(self):
        millis = str(int(round(time.time() * 1)))

        #Create SNS topic

        snsTopicName = "AmazonRekognitionExample" + millis

        topicResponse = self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

        attribs = self.sqs.get_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            AttributeNames = ['QueueArn']
        )['Attributes']

        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn = self.snsTopicArn,
            Protocol = 'sqs',
            Endpoint = sqsQueueArn)

        #Authorize SNS to write SQS queue 
        policy = """{{
            "Version":"2012-10-17",
            "Statement":[
            {{
              "Sid":"MyPolicy",
              "Effect":"Allow",
              "Principal" : {{"AWS" : "*"}},
              "Action":"SQS:SendMessage",
              "Resource": "{}",
              "Condition":{{
                "ArnEquals":{{
                  "aws:SourceArn": "{}"
                }}
              }}
            }}
            ]
        }}""".format(sqsQueueArn, self.snsTopicArn)

        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
                })
    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)

In [ ]:
def run_video_detection(roleArn, bucket, video):
    analyzer=VideoDetect(roleArn, bucket, video)
    analyzer.CreateTopicandQueue()
    
    analyzer.StartFaceDetection()
    if analyzer.GetSQSMessageSuccess()==True:
        results = analyzer.GetFaceDetectionResults()
    
    analyzer.DeleteTopicandQueue()
    return results

In [ ]:
def run_video_identification(roleArn, bucket, video, collection_id):
    analyzer=VideoDetect(roleArn, bucket, video)
    analyzer.CreateTopicandQueue()
    
    analyzer.StartFaceSearchCollection(collection_id)
    
    if analyzer.GetSQSMessageSuccess()==True:
        results = analyzer.GetFaceSearchCollectionResults()
    
    analyzer.DeleteTopicandQueue()
    return results

In [ ]:
def display_detected_faces(bucket, video, faces, ammend=False, label=False):
    obj = get_object(bucket, video, "../videos/" + video)
    
    vid_name = ".".join(video.split(".")[:-1])
        
    framecount = 0
    frame_array = []
    
    # gather the frames with boxes drawn around the faces
    vidcap = cv2.VideoCapture("../videos/" + video)
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    width  = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
    
        calced_timestamp = vidcap.get(cv2.CAP_PROP_POS_MSEC)
        if ret == False or calced_timestamp < 0 or (calced_timestamp==0 and framecount!=0):
            break
        
            
        h = frame.shape[0]
        w = frame.shape[1]
        c = frame.shape[2]
        
        detected = faces.get(str(int(calced_timestamp)), None)
        
        if detected is not None:
            print("Detected at: {}, {}".format(int(calced_timestamp), framecount))
            for face in detected:
                box = face['BoundingBox']

                left = w * box['Left']
                top = h * box['Top']
                right = (w * box['Width']) + left
                bottom = (h * box['Height']) + top

                start_point = (int(left), int(top)) 
                end_point = (int(right), int(bottom)) 

                # Blue color in BGR 
                color = (255, 0, 0) 

                # Line thickness of 2 px 
                thickness = 3 
                
                new_frame = cv2.rectangle(frame, start_point, end_point, color, thickness) 
                if label:
                    highest_confidence_emotion = None
                    highest_confidence = 0.0
                    for emotion in face['Emotions']:
                        if emotion['Confidence'] > highest_confidence:
                            highest_confidence = emotion['Confidence']
                            highest_confidence_emotion = emotion ['Type']

                    label= """Age Range: {}\nGender: {}\nSmile: {}\nEmotion: {}
                       """.format(
                           face['AgeRange'], 
                           "{} - {}".format(face['Gender']['Value'], '%.3f' % face['Gender']['Confidence']),
                           "{} - {}".format(face['Smile']['Value'], '%.3f' % face['Smile']['Confidence']),
                           "{} - {}".format(highest_confidence_emotion, '%.3f' % highest_confidence))

                    for i, line in enumerate(label.split('\n')):
                        y = int(bottom) + i*20
                        cv2.putText(
                            new_frame, 
                            line, 
                            (int(left), int(y)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            0.5,
                            (0,0,0),
                            2
                        )
                    for i, line in enumerate(label.split('\n')):
                        y = int(bottom) + i*20
                        cv2.putText(
                            new_frame, 
                            line, 
                            (int(left), int(y)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            0.5,
                            (255,255,255),
                            1
                        )
                frame_array.append(new_frame)
        else:
            if ammend:
                # img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # im_pil = Image.fromarray(img)
                
                is_success, im_buf_arr = cv2.imencode(".jpg", frame)
                byte_im = im_buf_arr.tobytes()
                # photo_bytes = cv2.imencode('.jpg', frame)[1].tobytes()

                ammend_faces = detect_faces_api(byte_im)
                
                colors = ['blue']*ammend_faces['face_count']
                bounding_boxes = []
                # gather the bounding boxes so we can easily access them 
                for face in ammend_faces['face_details']:
                    box = face['BoundingBox']

                    left = width * box['Left']
                    top = height * box['Top']
                    right = (width * box['Width']) + left
                    bottom = (height * box['Height']) + top

                    # set box params
                    start_point = (int(left), int(top)) 
                    end_point = (int(right), int(bottom)) 
                    color = (255, 0, 0) 
                    thickness = 2

                    # draw box on frames
                    new_frame = cv2.rectangle(frame, start_point, end_point, color, thickness) 
                    
                    if label:
                        highest_confidence_emotion = None
                        highest_confidence = 0.0
                        for emotion in face['Emotions']:
                            if emotion['Confidence'] > highest_confidence:
                                highest_confidence = emotion['Confidence']
                                highest_confidence_emotion = emotion ['Type'] 
                        label= """Age Range: {}\nGender: {}\nSmile: {}\nEmotion: {}
                           """.format(
                               face['AgeRange'], 
                               "{} - {}".format(face['Gender']['Value'], '%.3f' % face['Gender']['Confidence']),
                               "{} - {}".format(face['Smile']['Value'], '%.3f' % face['Smile']['Confidence']),
                               "{} - {}".format(highest_confidence_emotion, '%.3f' % highest_confidence))

                        for i, line in enumerate(label.split('\n')):
                            y = int(bottom) + i*20
                            cv2.putText(
                                new_frame, 
                                line, 
                                (int(left), int(y)), 
                                cv2.FONT_HERSHEY_DUPLEX, 
                                0.5,
                                (0,0,0),
                                2
                            )
                        for i, line in enumerate(label.split('\n')):
                            y = int(bottom) + i*20
                            cv2.putText(
                                new_frame, 
                                line, 
                                (int(left), int(y)), 
                                cv2.FONT_HERSHEY_DUPLEX, 
                                0.5,
                                (255,255,255),
                                1
                            )

                print("Ammended frame: {}".format(framecount))
                frame_array.append(new_frame)
            else:
                frame_array.append(frame)

        framecount+=1
    print(framecount)
    print(len(frame_array))
    # compile the frames into a video
    new_vid = "../videos/detected_labeled_" if label else "../videos/detected_"
    
    out = cv2.VideoWriter(
        new_vid + video, 
        cv2.VideoWriter_fourcc(*'DIVX'), 
        fps, 
        (w, h)
    )
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

    vidcap.release()
    cv2.destroyAllWindows()
    

In [ ]:
def estimate_crowd_size(bucket, video, faces, ammend=False, label_preamble='', detect_distinct=False):
    obj = get_object(bucket, video, "../videos/" + video)
    
    vid_name = ".".join(video.split(".")[:-1])
        
    framecount = 0
    frame_array = []
    
    # gather the frames with boxes drawn around the faces
    vidcap = cv2.VideoCapture("../videos/" + video)
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    width  = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
    
    # set up a collection for detect distinct
    if detect_distinct:
        delete_collection(vid_name)
        create_collection(vid_name)
        dest_folder="../images/cropped/"
        distinct_faces = 0
        identified_faces = {}
        bounding_boxes = []
        times = []
    
    prev_timestamp = 0
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        calced_timestamp = vidcap.get(cv2.CAP_PROP_POS_MSEC)
        
        if ret == False or calced_timestamp < 0 or (calced_timestamp==0 and framecount!=0):
            break
        
        h = frame.shape[0]
        w = frame.shape[1]
        c = frame.shape[2]
        
        detected = faces.get(str(int(calced_timestamp)), None)
                
        new_frame = frame.copy()
        if detected is not None:
            if not detect_distinct:
                label = '{}: {}'.format(label_preamble, len(detected))
                cv2.putText(
                    new_frame, 
                    label, 
                    (int(w/2), int(h*3/4)), 
                    cv2.FONT_HERSHEY_DUPLEX, 
                    1,
                    (0,0,0),
                    2)
                cv2.putText(
                    new_frame, 
                    label, 
                    (int(w/2), int(h*3/4)), 
                    cv2.FONT_HERSHEY_DUPLEX, 
                    1,
                    (255,255,255),
                    1)
                print('{}: {}'.format(framecount, label))
            else:
                is_success, im_buf_arr = cv2.imencode('.jpg', frame)
                byte_im = im_buf_arr.tobytes()
                
                photo_name = vid_name + '_cropped.jpg'
                cropped_faces = detect_faces_crop(photo_name, byte_im)
                
                bounding_boxes = []
                labels = []
                
                idx = 0
                for cropped_face in cropped_faces[0]:
                    existing = search_faces_by_image(
                        vid_name,
                        dest_folder + cropped_face,
                        70,
                        10
                    )
                    face_id = None
                    recorded = False
                    for e in existing:
                        if e['FaceId'] in identified_faces.keys():
                            recorded = True
                            break

                    if len(existing) == 0:
                        distinct_faces += 1
                        res = add_to_collection(
                            vid_name,
                            dest_folder + cropped_face,
                            1,
                            cropped_face
                        )
                        face_id = res[0][0]['FaceId']
                    elif recorded:
                        for e in existing:
                            if e['FaceId'] in identified_faces.keys():
                                face_id = e['FaceId']
                    else:
                        face_id = existing[0]['FaceId']
                        
                    if face_id is not None:
                        if face_id not in list(identified_faces.keys()):
                            identified_faces[face_id] = 0.0
                        else:
                            identified_faces[face_id] += (calced_timestamp-prev_timestamp)/1000
                    label = 'Time in video: {}'.format('%.3f' % identified_faces[face_id])
                    box = cropped_faces[1][idx]
                    
                    bounding_boxes.append(box)
                    labels.append(label)

                    left = w * box['Left']
                    top = h * box['Top']
                    right = (w * box['Width']) + left
                    bottom = (h * box['Height']) + top

                    start_point = (int(left), int(top)) 
                    end_point = (int(right), int(bottom)) 

                    # Blue color in BGR 
                    color = (255, 0, 0) 

                    # Line thickness of 2 px 
                    thickness = 3 

                    new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 
                    cv2.putText(
                        new_frame, 
                        label, 
                        (int(left), int(bottom)), 
                        cv2.FONT_HERSHEY_DUPLEX, 
                        1,
                        (255,0,0),
                        2)
                    cv2.putText(
                        new_frame, 
                        label, 
                        (int(left), int(bottom)), 
                        cv2.FONT_HERSHEY_DUPLEX, 
                        1,
                        (255,255,255),
                        1)
                    idx += 1
            frame_array.append(new_frame)
        else:
            if ammend:
                is_success, im_buf_arr = cv2.imencode('.jpg', frame)
                byte_im = im_buf_arr.tobytes()
                ammend_faces = detect_faces_api(byte_im)
                
                if not detect_distinct:
                    label = '{}: {}'.format(label_preamble, len(ammend_faces['face_details']))
                    cv2.putText(
                        new_frame, 
                        label, 
                        (int(w/2), int(h*3/4)), 
                        cv2.FONT_HERSHEY_DUPLEX, 
                        1,
                        (0,0,0),
                        2)
                    cv2.putText(
                        new_frame, 
                        label, 
                        (int(w/2), int(h*3/4)), 
                        cv2.FONT_HERSHEY_DUPLEX, 
                        1,
                        (255,255,255),
                        1)
                    print('Ammended {} label: {}'.format(framecount, label))
                else:
                    photo_name = vid_name + '_cropped.jpg'
                    cropped_faces = detect_faces_crop(photo_name, byte_im)
                    
                    idx = 0
                    for cropped_face in cropped_faces[0]:
                        existing = search_faces_by_image(
                            vid_name,
                            dest_folder + cropped_face,
                            70,
                            10
                        )
                        face_id = None
                        recorded = False
                        for e in existing:
                            if e['FaceId'] in identified_faces.keys():
                                recorded = True
                                break
                        
                        if len(existing) == 0:
                            distinct_faces += 1
                            res = add_to_collection(
                                vid_name,
                                dest_folder + cropped_face,
                                1,
                                cropped_face
                            )
                            face_id = res[0][0]['FaceId']
                        elif recorded:
                            for e in existing:
                                if e['FaceId'] in identified_faces.keys():
                                    face_id = e['FaceId']
                        else:
                            face_id = existing[0]['FaceId']

                        if face_id is not None:
                            if face_id not in list(identified_faces.keys()):
                                identified_faces[face_id] = 0.0
                            else:
                                identified_faces[face_id] += (calced_timestamp-prev_timestamp)/1000
                        label = 'Time in video: {}'.format('%.3f' % identified_faces[face_id])
                        box = cropped_faces[1][idx]

                        left = w * box['Left']
                        top = h * box['Top']
                        right = (w * box['Width']) + left
                        bottom = (h * box['Height']) + top

                        start_point = (int(left), int(top)) 
                        end_point = (int(right), int(bottom)) 

                        # Blue color in BGR 
                        color = (255, 0, 0) 

                        # Line thickness of 2 px 
                        thickness = 3 

                        new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 
                        cv2.putText(
                            new_frame, 
                            label, 
                            (int(left), int(bottom)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            1,
                            (255,0,0),
                            2)
                        cv2.putText(
                            new_frame, 
                            label, 
                            (int(left), int(bottom)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            1,
                            (255,255,255),
                            1)
                        idx += 1
                frame_array.append(new_frame)
            else:
                if detect_distinct:
                    box_idx = 0
                    for box in bounding_boxes:
                        
                        label = labels[box_idx]
                        left = w * box['Left']
                        top = h * box['Top']
                        right = (w * box['Width']) + left
                        bottom = (h * box['Height']) + top

                        start_point = (int(left), int(top)) 
                        end_point = (int(right), int(bottom)) 

                        # Blue color in BGR 
                        color = (255, 0, 0) 

                        # Line thickness of 2 px 
                        thickness = 3 

                        new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 
                        cv2.putText(
                            new_frame, 
                            label, 
                            (int(left), int(bottom)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            1,
                            (255,0,0),
                            2)
                        cv2.putText(
                            new_frame, 
                            label, 
                            (int(left), int(bottom)), 
                            cv2.FONT_HERSHEY_DUPLEX, 
                            1,
                            (255,255,255),
                            1)
                        box_idx += 1
                    frame_array.append(new_frame)
                else:
                    frame_array.append(frame)

        framecount+=1
        prev_timestamp = calced_timestamp
        if detect_distinct:
            print('identified_faces at {}: {}'.format(str(int(calced_timestamp)), identified_faces))
    
    if detect_distinct:
        print('------------------')
        print('Detected {} distinct faces'.format(distinct_faces))
        print('identified_faces: {}'.format(identified_faces))
        print('------------------')
        delete_collection(vid_name)
                        
    # compile the frames into a video
    new_vid = '../videos/detected_crowd_count_'
    
    if len(frame_array) > 0:
        out = cv2.VideoWriter(
            new_vid + video, 
            cv2.VideoWriter_fourcc(*'DIVX'), 
            fps, 
            (w, h)
        )
        for i in range(len(frame_array)):
            out.write(frame_array[i])
            
    out.release()
        
    vidcap.release()
    cv2.destroyAllWindows()

In [ ]:
def display_identification_results(video, bucket, faces, collection_id):
    obj = get_object(bucket, video, "../videos/" + video)
    
    vid_name = ".".join(video.split(".")[:-1])
        
    framecount = 0
    frame_array = []
    
    # gather the frames with boxes drawn around the faces
    vidcap = cv2.VideoCapture("../videos/" + video)
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    width  = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
    
    authorized = {}
    unauthorized = 0
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        
        calced_timestamp = vidcap.get(cv2.CAP_PROP_POS_MSEC)
        
        if ret == False or calced_timestamp < 0 or (calced_timestamp==0 and framecount!=0):
            break
        
        h = frame.shape[0]
        w = frame.shape[1]
        c = frame.shape[2]
        
        detected = faces.get(str(int(calced_timestamp)), None)
        
        if detected is not None:    
            print("Detected at: {}".format(calced_timestamp))
            for people in detected:
                for person in people:
                    face_id = people[person]['FaceId']
                    
                    if face_id is None:
                        unauthorized += 1
                    else:
                        # make sure we get only the distinct name
                        face_name = search_faces(collection_id, face_id)[0]['ExternalImageId'].split('_')[0]
                        authorized[face_name] = people[person]['Details']['BoundingBox']
        
        if detected is not None:
            unauthorized = len(detected) - len(authorized)
        
        new_frame = frame.copy()
        for face_id in authorized:
            box = authorized[face_id]

            left = w * box['Left']
            top = h * box['Top']
            right = (w * box['Width']) + left
            bottom = (h * box['Height']) + top

            start_point = (int(left), int(top)) 
            end_point = (int(right), int(bottom)) 

            # Blue color in BGR 
            color = (255, 0, 0) 

            # Line thickness of 2 px 
            thickness = 3 

            new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 
        
        label = 'Authorized: {}'.format(' '.join(list(authorized.keys())))
        cv2.putText(
            new_frame, 
            label, 
            (int(w/4), int(h*3/4)), 
            cv2.FONT_HERSHEY_DUPLEX, 
            0.5,
            (255,0,0),
            2)
        cv2.putText(
            new_frame, 
            label, 
            (int(w/4), int(h*3/4)), 
            cv2.FONT_HERSHEY_DUPLEX, 
            0.5,
            (255,255,255),
            1)
        frame_array.append(new_frame)
        
        label = 'Unauthorized: {}'.format(unauthorized)
        cv2.putText(
            new_frame, 
            label, 
            (int(w/4), int(h*3/4) + 20), 
            cv2.FONT_HERSHEY_DUPLEX, 
            0.5,
            (0,0,255),
            2)
        cv2.putText(
            new_frame, 
            label, 
            (int(w/4), int(h*3/4) + 20), 
            cv2.FONT_HERSHEY_DUPLEX, 
            0.5,
            (255,255,255),
            1)
        frame_array.append(new_frame)
            
        framecount += 1
        
    print(authorized)
                        
    # compile the frames into a video
    new_vid = '../videos/detect_'
    
    out = cv2.VideoWriter(
        new_vid + video, 
        cv2.VideoWriter_fourcc(*'DIVX'), 
        fps, 
        (w, h)
    )
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

    vidcap.release()
    cv2.destroyAllWindows()

In [ ]:
def estimate_capacity(bucket, video, faces, ammend=False, capacity=5):
    obj = get_object(bucket, video, "../videos/" + video)
    
    vid_name = ".".join(video.split(".")[:-1])
        
    framecount = 0
    frame_array = []
    
    # gather the frames with boxes drawn around the faces
    vidcap = cv2.VideoCapture("../videos/" + video)
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    width  = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) 
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
    
        calced_timestamp = vidcap.get(cv2.CAP_PROP_POS_MSEC)
        if ret == False or calced_timestamp < 0 or (calced_timestamp==0 and framecount!=0):
            break
        
            
        h = frame.shape[0]
        w = frame.shape[1]
        c = frame.shape[2]
        
        detected = faces.get(str(int(calced_timestamp)), None)
        
        new_frame = frame.copy()
        if detected is not None:
            print("Detected at: {}, {}".format(int(calced_timestamp), framecount))
            for face in detected:
                box = face['BoundingBox']

                left = w * box['Left']
                top = h * box['Top']
                right = (w * box['Width']) + left
                bottom = (h * box['Height']) + top

                start_point = (int(left), int(top)) 
                end_point = (int(right), int(bottom)) 

                # Blue color in BGR 
                color = (255, 0, 0) 

                # Line thickness of 2 px 
                thickness = 3 
                
                new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 

            label = 'Capacity: {}'.format(capacity - len(detected))
            cv2.putText(
                new_frame, 
                label, 
                (40, 40), 
                cv2.FONT_HERSHEY_DUPLEX, 
                2,
                (0,0,0),
                3
            )
            frame_array.append(new_frame)
        else:
            if ammend:
                # img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # im_pil = Image.fromarray(img)
                
                is_success, im_buf_arr = cv2.imencode(".jpg", new_frame)
                byte_im = im_buf_arr.tobytes()
                # photo_bytes = cv2.imencode('.jpg', frame)[1].tobytes()

                ammend_faces = detect_faces_api(byte_im)
                
                colors = ['blue']*ammend_faces['face_count']
                bounding_boxes = []
                # gather the bounding boxes so we can easily access them 
                for face in ammend_faces['face_details']:
                    box = face['BoundingBox']

                    left = width * box['Left']
                    top = height * box['Top']
                    right = (width * box['Width']) + left
                    bottom = (height * box['Height']) + top

                    # set box params
                    start_point = (int(left), int(top)) 
                    end_point = (int(right), int(bottom)) 
                    color = (255, 0, 0) 
                    thickness = 2

                    # draw box on frames
                    new_frame = cv2.rectangle(new_frame, start_point, end_point, color, thickness) 
                    
                label = 'Capacity: {}'.format(capacity - len(ammend_faces))
                cv2.putText(
                    new_frame, 
                    label,
                    (40, 40), 
                    cv2.FONT_HERSHEY_DUPLEX, 
                    2,
                    (0,0,0),
                    3
                )
                frame_array.append(new_frame)
                print("Ammended frame: {}".format(framecount))
            else:
                frame_array.append(frame)

        framecount+=1
    # compile the frames into a video
    new_vid = "../videos/detected_labeled_" if label else "../videos/detected_"
    
    out = cv2.VideoWriter(
        new_vid + video, 
        cv2.VideoWriter_fourcc(*'DIVX'), 
        fps, 
        (w, h)
    )
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

    vidcap.release()
    cv2.destroyAllWindows()
    